In [1]:
from LassoLayer import LassoLayer
import pandas as pd

In [2]:
df = pd.read_pickle('titanic.pkl')
input_df = df.drop('Survived', axis=1)
output_df = df['Survived']
input_df.shape

(891, 18)

In [17]:
# basic model from https://www.kaggle.com/vincentlugat/titanic-neural-networks-keras-81-8
# with the lasso input layer added to illustrate the concept

# all I did was add the Lasso Layer for illustration purposes
from keras.layers import Input, Dense, LeakyReLU, Layer, Dropout
from keras.models import Model
import keras.backend as K
from keras import regularizers
from keras.losses import categorical_crossentropy, binary_crossentropy, hinge, squared_hinge
from keras import backend as K
from keras.layers import Layer
from keras.optimizers import Adam
import tensorflow as tf

inputs = Input(shape=(input_df.shape[1],))
lasso = LassoLayer(input_df.shape[1], kernel_regularizer=tf.keras.regularizers.l1(0.01))(inputs)
dense1 = Dense(13, activation='relu')(lasso)
dropout = Dropout(0.2)(dense1)
dense2 = Dense(8, activation='relu')(dropout)
# this was added in addition to the original model
# for the purpose of the embedding
# the name is specified so we can build an intermediate model to
# extract the output
dense3 = Dense(2, activation='relu', name='embedding')(dense2)
output = Dense(1, activation='sigmoid')(dense3)
model = Model(inputs=inputs, outputs=output)
model.compile(loss='binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.fit(input_df, output_df,epochs = 20, batch_size = 10)

Epoch 1/20
90/90 [==============================] - 1s 13ms/step - loss: 0.6857 - accuracy: 0.6162
Epoch 2/20
90/90 [==============================] - 1s 8ms/step - loss: 0.6581 - accuracy: 0.6162
Epoch 3/20
90/90 [==============================] - 1s 6ms/step - loss: 0.6087 - accuracy: 0.6162
Epoch 4/20
90/90 [==============================] - 0s 5ms/step - loss: 0.5667 - accuracy: 0.6162
Epoch 5/20
90/90 [==============================] - 0s 4ms/step - loss: 0.5590 - accuracy: 0.6835
Epoch 6/20
90/90 [==============================] - 0s 5ms/step - loss: 0.5486 - accuracy: 0.7744
Epoch 7/20
90/90 [==============================] - 0s 4ms/step - loss: 0.5456 - accuracy: 0.7868
Epoch 8/20
90/90 [==============================] - 0s 4ms/step - loss: 0.5419 - accuracy: 0.7845
Epoch 9/20
90/90 [==============================] - 1s 8ms/step - loss: 0.5384 - accuracy: 0.7901
Epoch 10/20
90/90 [==============================] - 0s 4ms/step - loss: 0.5367 - accuracy: 0.7868
Epoch 11/20
90/90 

In [19]:
intermediate_layer_model = Model(inputs=model.input, outputs=model.get_layer('embedding').output)
embedding = intermediate_layer_model.predict(input_df)

In [21]:
import matplotlib.pl

(891, 2)